In [2]:
import os
import re
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DeepLake


In [1]:
text = """
Objection: "There's no money."
It could be that your prospect's business simply isn't big enough or generating enough cash right now to afford a product like yours. Track their growth and see how you can help your prospect get to a place where your offering would fit into their business.

Objection: "We don't have any budget left this year."
A variation of the "no money" objection, what your prospect's telling you here is that they're having cash flow issues. But if there's a pressing problem, it needs to get solved eventually. Either help your prospect secure a budget from executives to buy now or arrange a follow-up call for when they expect funding to return.

Objection: "We need to use that budget somewhere else."
Prospects sometimes try to earmark resources for other uses. It's your job to make your product/service a priority that deserves budget allocation now. Share case studies of similar companies that have saved money, increased efficiency, or had a massive ROI with you.
"""

# Split the text into a list using the keyword "Objection: "
objections_list = text.split("Objection: ")[1:]  # We ignore the first split as it is empty
# Now, prepend "Objection: " to each item as splitting removed it
objections_list = ["Objection: " + objection for objection in objections_list]
print(objections_list)

['Objection: "There\'s no money."\nIt could be that your prospect\'s business simply isn\'t big enough or generating enough cash right now to afford a product like yours. Track their growth and see how you can help your prospect get to a place where your offering would fit into their business.\n\n', 'Objection: "We don\'t have any budget left this year."\nA variation of the "no money" objection, what your prospect\'s telling you here is that they\'re having cash flow issues. But if there\'s a pressing problem, it needs to get solved eventually. Either help your prospect secure a budget from executives to buy now or arrange a follow-up call for when they expect funding to return.\n\n', 'Objection: "We need to use that budget somewhere else."\nProspects sometimes try to earmark resources for other uses. It\'s your job to make your product/service a priority that deserves budget allocation now. Share case studies of similar companies that have saved money, increased efficiency, or had a m

In [3]:
from dotenv import load_dotenv
load_dotenv('/Users/srishtysuman/.env')

True

In [6]:
# We’re going to define a class that handles the database creation, database loading, and database querying.

class DeepLakeLoader:
    def __init__(self, source_data_path):
        self.data = self.split_data()
        self.dataset_path=source_data_path
        if False:
         self.db = self.load_db()
        else:
            self.db = self.create_db()


    def split_data(self):  
        """  
        Preprocess the data by splitting it into passages.  

        If using a different data source, this function will need to be modified.  

        Returns:  
            split_data (list): List of passages.  
        """  
        with open('/Users/srishtysuman/LearnLLMs/1_LangChainAndVectorDatabases/4_KeepingknowledgeOrganizedWithIndexes/7_AISalesAssistant/objections.txt', 'r') as f:  
            content = f.read()  
        split_data = re.split(r'(?=\d+\. )', content)
        if split_data[0] == '':  
            split_data.pop(0)  
        split_data = [entry for entry in split_data if len(entry) >= 30]  
        return split_data

    def load_db(self):  
        """  
        Load the database if it already exists.  

        Returns:  
            DeepLake: DeepLake object.  
        """  
        return DeepLake(dataset_path=self.dataset_path, embedding_function=OpenAIEmbeddings(), read_only=True)  

    def create_db(self):  
        """  
        Create the database if it does not already exist.  

        Databases are stored in the deeplake directory.  

        Returns:  
            DeepLake: DeepLake object.  
        """  
        return DeepLake.from_texts(self.data, OpenAIEmbeddings(), dataset_path=self.dataset_path)

    def query_db(self, query):  
        """  
        Query the database for passages that are similar to the query.  

        Args:  
            query (str): Query string.  

        Returns:  
            content (list): List of passages that are similar to the query.  
        """  
        results = self.db.similarity_search(query, k=3)  
        content = []  
        for result in results:  
            content.append(result.page_content)  
        return content

db = DeepLakeLoader('https:/app.activeloop.ai/srishtysuman2919/ai_sales_assistant')

results = db.query_db("We need to use that budget somewhere else.")


from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo")
system_message = SystemMessage(content="You are a Sales Call Assistant who detects potential objections from the customer (e.g. “It’s too expensive” or “The product doesn’t work for us”) and provide well-informed recommendations to the salesperson on how best to handle them.")
human_message = HumanMessage(content=f'Customer objection: {objections_list} | Relevant guidelines: {results}')

response = chat([system_message, human_message])

print(response.content)





/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
Creating 1 embeddings in 1 batches of size 1:: 100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


Dataset(path='https:/app.activeloop.ai/srishtysuman2919/ai_sales_assistant', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (1, 1)      str     None   
 metadata     json      (1, 1)      str     None   
 embedding  embedding  (1, 1536)  float32   None   
    id        text      (1, 1)      str     None   


/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


When faced with the objection "There's no money," it's important to understand the prospect's financial situation. If their business is not currently generating enough cash to afford your product, you can track their growth and find ways to help them get to a point where your offering would fit into their budget. Offer alternative payment options such as financing or a payment plan to make it more affordable for them.

If the objection is "We don't have any budget left this year," it indicates cash flow issues. However, if there is a pressing problem that needs to be solved, it should eventually be addressed. You can help the prospect by assisting them in securing a budget from executives to make the purchase now or suggest arranging a follow-up call for when they expect funding to become available again.

When the objection is "We need to use that budget somewhere else," it means that the prospect is trying to allocate resources to other areas. In this case, you need to demonstrate th